# Lib import

In [39]:
import pandas as pd
import numpy as np
import scipy.stats as sps
import seaborn as sns

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d  
from matplotlib import cm

# Загрузка данных и просмотр их первоначальной структуры

Монтируем гугл-диск, чтобы загрузить датасет:

In [40]:
from google.colab import drive
drive.mount('/content/drive/')

%cd /content/drive/My Drive/ML_Data/Data_Raw/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/ML_Data/Data_Raw


"Читаем" датасет:

In [41]:
filename = 'Data_Test.txt'
df = pd.read_fwf(filename)

In [42]:
df.head()

,Beam length (um),Beam width (nm),Thickness_1 (nm),Thickness_2 (nm),Temperature (K),Distance (nm),Gate voltage (V),Eigenfrequency (Hz),Quality factor,Effective mass (kg),Noise (kg^2/s^3),TED (W)
0,1.0,50.000000,50.000000,5.000000,0.010000,10.000000,0.000100,2.803039857612273E9+467173.2966291722i,3000.000083,4.463888e-18,2.303375e-24,4.907176e-13
1,1.0,50.000000,50.000000,5.000000,0.010000,10.000000,0.000100,4.761706392958552E9+793617.7101129907i,3000.000083,3.215783e-18,2.818846e-24,2.061557e-09
2,1.0,50.000000,50.000000,5.000000,0.010000,10.000000,0.000100,2.3686217100034855E10+3947715.782323136i,2999.990172,1.484737e-18,6.473942e-24,9.672427e-13
3,1.0,50.000000,50.000000,5.000000,0.010000,10.000000,0.000100,3.611185374160151E10+6601533.156939676i,2735.111139,3.025101e-19,2.205758e-24,4.907778e-13
4,2.0,73.684211,68.421053,7.894737,15.798947,62.105263,3.157989,1.3956162769663472E9+157837.04112249007i,4421.067029,1.825222e-17,1.256800e-21,6.382872e-13


# Функции для обработки датасета

Функция, реализующая трансформацию датасета 4row x 1col --> 1row x 4col:

In [43]:
def process_data(df, len_common):
    # num_common - количество стоблцов, одинаковых для всех мод
    data = []
    for index in range(df.shape[0]):
        if index % 4 == 0:
            i = 0
            row = df.iloc[index].values.flatten().tolist()
        else:
            row += df.iloc[index, len_common:].values.flatten().tolist()
        i += 1
        if i == 4:
            data.append(row)
    return data

Функция, реализующая замену индексовых имен колонок на буквенные:

In [44]:
def name_data_columns(df, data, len_common):
    #len_common - число колонок, общих для всех резонансных мод
    len_differ = df.shape[1] - len_common # число различающихся колонок

    cols_common = list(df.columns)[:len_common]
    cols_differ = list(df.columns)[len_common:]
    print(len_differ, len(cols_differ))

    columns = [] # сюда будем записывать финальный вариант колонок.
    # Прим.: pandas не дает изменять колонки внутри цикла,
    # их можно изменить только единственным присваиванием
    
    # проименовываем общие колонки:
    for col_index in range(0, len_common):
        columns.append(cols_common[col_index])
    
    # проименовываем различающиеся колонки:
    for mode_index in range(1, 5):
        for col_index in range(0, len_differ):
            columns.append(f'M{mode_index} ' + cols_differ[col_index])
    data.columns = columns

    return data

Функция, возвращающая индексы строк (и столбцов, если рассматривать разные моды колебаний), в которых детектирована аномалия $ Im\left(f_0\right) < 0$:

In [45]:
def return_neg_frequencies_index(df, freq_indices):
    list_i = []
    list_j = []
    for j in freq_indices:
        for i in range(df.shape[0]):
            if '-' in df.iloc[i,j]:
                list_i.append(i)
                list_j.append(j)
    return list_i, list_j

Функция, реализующая удаление описанных выше аномалий:

In [46]:
def del_neg_frequencies(df, freq_indices):
    lst_i, _ = return_neg_frequencies_index(data, freq_indices)
    lst_i.sort(reverse=True)
    for i in lst_i:
        df.drop(index=i, inplace=True)
    return df

Функция, реализующая удаление мнимой части резонансной частоты для всех элементов датасета:

In [47]:
def del_im_frequency(df, freq_indices):
    for j in freq_indices:
        for i in range(0, df.shape[0]):
            df.iloc[i, j] = float(str(df.iloc[i, j]).split('+', 1)[0])
    return df

Функция, реализующая округление чисел до `decimals` знаков после запятой (потому что метод конечных элементов иногда, например, вместо числа `200` записывает `200.0000000111`):

In [48]:
def round_data_float(data, decimals):
    return data.round(decimals=decimals)

Функция, реализующая округление значений до типа `int`, которые должны были быть записаны методом конечных элементов как `int`, но были записаны как `float`:

In [49]:
def round_data_int(data, cols_indices_to_round):
    for col_index in cols_indices_to_round:
        data.iloc[:, col_index] = data.iloc[:, col_index].astype(int)

# Обработка датасета

Преобразование 4row x 1col --> 1row x 4col:

In [50]:
data = pd.DataFrame(process_data(df, len_common=7))
data.head()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,1.0,50.000000,50.000000,5.000000,0.010000,10.000000,0.000100,2.803039857612273E9+467173.2966291722i,3000.000083,4.463888e-18,...,2.3686217100034855E10+3947715.782323136i,2999.990172,1.484737e-18,6.473942e-24,9.672427e-13,3.611185374160151E10+6601533.156939676i,2735.111139,3.025101e-19,2.205758e-24,4.907778e-13
1,2.0,73.684211,68.421053,7.894737,15.798947,62.105263,3.157989,1.3956162769663472E9+157837.04112249007i,4421.067029,1.825222e-17,...,1.7177697287768831E10+1944530.8599443021i,4416.925862,6.114080e-18,5.186656e-21,2.467168e-12,2.5415038964868347E10-9.079290791735753E7i,139.961587,1.272042e-18,5.038457e-20,3.491894e-12
2,3.0,97.368421,86.842105,10.789474,31.587895,114.210526,6.315879,9.28138394816084E8+79435.27010549096i,5842.105110,4.626296e-17,...,1.3572067337696001E10-4.005563058471869E7i,169.415225,1.504223e-17,2.335764e-19,4.868676e-12,1.5407651265497349E10+6.61568129504752E8i,11.644796,1.368365e-17,3.512593e-18,1.295388e-12
3,4.0,121.052632,105.263158,13.684211,47.376842,166.315789,9.473768,6.949824286034456E8+47860.32876622244i,7260.527106,9.340537e-17,...,6.355574824087556E9+3.125752905246249E9i,1.016647,3.301764e-17,3.761489e-17,1.176682e-12,7.653863140186178E9+2.3317784995471174E8i,16.412072,2.965607e-17,2.262673e-18,1.262608e-12
4,5.0,144.736842,123.684211,16.578947,63.165789,218.421053,12.631658,5.553439664042764E8+31973.79808686548i,8684.360314,1.645929e-16,...,9.42988916937626E9+585311.630059936i,8055.443191,5.559638e-17,9.081318e-21,1.035849e-11,1.352728882491788E10+1043120.6704471351i,6484.047919,1.158770e-17,3.373238e-21,1.819410e-11


Форматирование названий колонок:

In [51]:
data = name_data_columns(df, data, len_common=7)
data.head()

5 5


,Beam length (um),Beam width (nm),Thickness_1 (nm),Thickness_2 (nm),Temperature (K),Distance (nm),Gate voltage (V),M1 Eigenfrequency (Hz),M1 Quality factor,M1 Effective mass (kg),...,M3 Eigenfrequency (Hz),M3 Quality factor,M3 Effective mass (kg),M3 Noise (kg^2/s^3),M3 TED (W),M4 Eigenfrequency (Hz),M4 Quality factor,M4 Effective mass (kg),M4 Noise (kg^2/s^3),M4 TED (W)
0,1.0,50.000000,50.000000,5.000000,0.010000,10.000000,0.000100,2.803039857612273E9+467173.2966291722i,3000.000083,4.463888e-18,...,2.3686217100034855E10+3947715.782323136i,2999.990172,1.484737e-18,6.473942e-24,9.672427e-13,3.611185374160151E10+6601533.156939676i,2735.111139,3.025101e-19,2.205758e-24,4.907778e-13
1,2.0,73.684211,68.421053,7.894737,15.798947,62.105263,3.157989,1.3956162769663472E9+157837.04112249007i,4421.067029,1.825222e-17,...,1.7177697287768831E10+1944530.8599443021i,4416.925862,6.114080e-18,5.186656e-21,2.467168e-12,2.5415038964868347E10-9.079290791735753E7i,139.961587,1.272042e-18,5.038457e-20,3.491894e-12
2,3.0,97.368421,86.842105,10.789474,31.587895,114.210526,6.315879,9.28138394816084E8+79435.27010549096i,5842.105110,4.626296e-17,...,1.3572067337696001E10-4.005563058471869E7i,169.415225,1.504223e-17,2.335764e-19,4.868676e-12,1.5407651265497349E10+6.61568129504752E8i,11.644796,1.368365e-17,3.512593e-18,1.295388e-12
3,4.0,121.052632,105.263158,13.684211,47.376842,166.315789,9.473768,6.949824286034456E8+47860.32876622244i,7260.527106,9.340537e-17,...,6.355574824087556E9+3.125752905246249E9i,1.016647,3.301764e-17,3.761489e-17,1.176682e-12,7.653863140186178E9+2.3317784995471174E8i,16.412072,2.965607e-17,2.262673e-18,1.262608e-12
4,5.0,144.736842,123.684211,16.578947,63.165789,218.421053,12.631658,5.553439664042764E8+31973.79808686548i,8684.360314,1.645929e-16,...,9.42988916937626E9+585311.630059936i,8055.443191,5.559638e-17,9.081318e-21,1.035849e-11,1.352728882491788E10+1043120.6704471351i,6484.047919,1.158770e-17,3.373238e-21,1.819410e-11


Вывод информации об аномалиях мнимой части резонансной частоты, их удаление:

In [52]:
frequency_indices = [7, 12, 17, 22] # индексы колонок, в которых содержатся значения резонансных частот

# Детектирование аномалий:
lst_i, lst_j = return_neg_frequencies_index(data, frequency_indices)
print("Row and column indices of detected anomalies of imaginary part of resonant frequency:")
print(lst_i, lst_j)
print(f"The length of the dataset is {data.shape[0]}")

# Удаление аномалий:
data = del_neg_frequencies(data, frequency_indices)
lst_i, lst_j = return_neg_frequencies_index(data, frequency_indices)
print(f"Anomalies have been deleted, the length of the dataset is {data.shape[0]}")

Row and column indices of detected anomalies of imaginary part of resonant frequency:
[2, 1, 6, 7, 8, 9] [17, 22, 22, 22, 22, 22]
The length of the dataset is 20
Anomalies have been deleted, the length of the dataset is 14


Удаление мнимой части всех элементов датасета:

In [53]:
data = del_im_frequency(data, [7, 12, 17, 22])
data.head()

,Beam length (um),Beam width (nm),Thickness_1 (nm),Thickness_2 (nm),Temperature (K),Distance (nm),Gate voltage (V),M1 Eigenfrequency (Hz),M1 Quality factor,M1 Effective mass (kg),...,M3 Eigenfrequency (Hz),M3 Quality factor,M3 Effective mass (kg),M3 Noise (kg^2/s^3),M3 TED (W),M4 Eigenfrequency (Hz),M4 Quality factor,M4 Effective mass (kg),M4 Noise (kg^2/s^3),M4 TED (W)
0,1.0,50.000000,50.000000,5.000000,0.010000,10.000000,0.000100,2803039857.612273,3000.000083,4.463888e-18,...,23686217100.034855,2999.990172,1.484737e-18,6.473942e-24,9.672427e-13,36111853741.601509,2735.111139,3.025101e-19,2.205758e-24,4.907778e-13
3,4.0,121.052632,105.263158,13.684211,47.376842,166.315789,9.473768,694982428.603446,7260.527106,9.340537e-17,...,6355574824.087556,1.016647,3.301764e-17,3.761489e-17,1.176682e-12,7653863140.186178,16.412072,2.965607e-17,2.262673e-18,1.262608e-12
4,5.0,144.736842,123.684211,16.578947,63.165789,218.421053,12.631658,555343966.404276,8684.360314,1.645929e-16,...,9429889169.376261,8055.443191,5.559638e-17,9.081318e-21,1.035849e-11,13527288824.917879,6484.047919,1.158770e-17,3.373238e-21,1.819410e-11
5,6.0,168.421053,142.105263,19.473684,78.954737,270.526316,15.789547,462380543.580419,10105.362883,2.647434e-16,...,8198813092.682148,8122.057461,8.954131e-17,1.094781e-20,1.412529e-11,11702674805.834019,8222.568361,1.873763e-17,3.230065e-21,2.537991e-11
10,70.0,286.842105,234.210526,33.947368,157.899474,531.052632,31.578995,1560620.948014,17207.934873,7.023254e-15,...,5561491.324551,17201.322648,6.708401e-15,3.859902e-24,1.096633e-11,8664294.867125,17202.547397,7.799296e-15,6.990750e-24,1.547307e-11


Дополнительно удалим строку с индексом `2`, потому что в ячейке `[2, 'M1 TED (W)']` содержится аномалия в данных (неопознанное вторжение символа `s` в `float`-значение) (поймано на графиках распределений):

In [54]:
#data = data.drop(2)
#print(df.iloc[0:3, 17])

In [55]:
data.describe()

,Beam length (um),Beam width (nm),Thickness_1 (nm),Thickness_2 (nm),Temperature (K),Distance (nm),Gate voltage (V),M1 Quality factor,M1 Effective mass (kg),M1 Noise (kg^2/s^3),...,M2 Noise (kg^2/s^3),M2 TED (W),M3 Quality factor,M3 Effective mass (kg),M3 Noise (kg^2/s^3),M3 TED (W),M4 Quality factor,M4 Effective mass (kg),M4 Noise (kg^2/s^3),M4 TED (W)
count,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,1.400000e+01,1.400000e+01,...,1.400000e+01,1.400000e+01,14.000000,1.400000e+01,1.400000e+01,1.400000e+01,14.000000,1.400000e+01,1.400000e+01,1.400000e+01
mean,1034.714286,315.601504,256.578947,37.462406,177.071767,594.323308,35.413575,18923.601394,2.413237e-13,3.143973e-22,...,4.849857e-22,4.522637e-06,18221.225069,2.502773e-13,2.688209e-18,6.150541e-11,18099.017534,2.531605e-13,1.620919e-19,6.561146e-11
std,1088.616160,143.067161,111.274458,17.485986,95.374928,314.747754,19.075590,8567.713779,2.811515e-13,6.240968e-22,...,9.635296e-22,9.481210e-06,9698.334858,2.919512e-13,1.005259e-17,1.981199e-10,9855.537921,2.959491e-13,6.045899e-19,1.953438e-10
min,1.000000,50.000000,50.000000,5.000000,0.010000,10.000000,0.000100,3000.000083,4.463888e-18,1.145715e-27,...,1.740595e-27,1.933494e-12,1.016647,1.484737e-18,2.328940e-27,9.672427e-13,16.412072,3.025101e-19,2.924100e-27,4.907778e-13
25%,22.000000,198.026316,165.131579,23.092105,98.690921,335.657895,19.736909,11881.005880,1.954371e-15,2.073040e-27,...,4.148544e-27,2.481147e-12,10391.873758,1.744256e-15,6.227859e-27,3.937172e-12,10467.563120,1.972066e-15,8.313482e-27,6.454650e-12
50%,725.000000,346.052632,280.263158,41.184211,197.371842,661.315789,39.473718,20762.851195,1.226937e-13,1.506782e-26,...,3.017522e-26,7.213949e-12,20761.811166,1.227088e-13,4.535756e-26,6.811453e-12,20763.196185,1.227512e-13,6.067718e-26,1.121587e-11
75%,1862.500000,423.026316,340.131579,50.592105,248.685921,830.657895,49.736859,25384.544122,4.655202e-13,2.010915e-24,...,2.736385e-24,1.735246e-09,25380.973552,4.655528e-13,5.820432e-24,1.081437e-11,25380.297281,4.655935e-13,5.794502e-24,1.751384e-11
max,3000.000000,500.000000,400.000000,60.000000,300.000000,1000.000000,60.000000,29834.677320,8.274242e-13,1.468657e-21,...,2.378416e-21,2.902176e-05,29887.471445,8.278642e-13,3.761489e-17,7.492188e-10,29895.736550,8.281712e-13,2.262673e-18,7.428655e-10


Дополнительно поймаем аномалию, связанную с сильно заниженным значением термоупругих потерь четвертой моды (поймано на графиках распределений). Одновременно столбец резонансных частот конвертируется в тип `float`.

In [56]:
def catch_parameter_overflow(df, param_name, lower_limit, upper_limit):
    mask = (df[param_name].astype(float) > lower_limit) & (df[param_name].astype(float) < upper_limit)
    return df[mask]

def catch_overflow(df, param_limits):

    # Это кусок кода на случай, если на рез. частоту ограничений не накладывается, 
    #   но в любом случае надо конвертировать данные в тип float
    if 'Eigenfrequency (Hz)' not in param_limits.keys():
        for mode_number in range(1, 5):
            df.loc[:, f'M{mode_number} ' + 'Eigenfrequency (Hz)'] = df.loc[:, f'M{mode_number} ' + 'Eigenfrequency (Hz)'].astype(float)

    for param in param_limits.keys():
        for mode_number in range(1, 5):

            # Преобразуем тип резонансной частоты в тип float
            if param == 'Eigenfrequency (Hz)':
                df.loc[:, f'M{mode_number} ' + param] = df.loc[:, f'M{mode_number} ' + param].astype(float)

            # Последовательно накладываем маски ограничений
            df = catch_parameter_overflow(df, param_name=f'M{mode_number} ' + param,
                                          lower_limit=param_limits[param][0],
                                          upper_limit=param_limits[param][1])
    return df

In [57]:
param_limits = {#'Eigenfrequency (Hz)': [0, 3e7], 
                #'Quality factor': [0, 75000], 
                'TED (W)': [1e-20, 1]}

print('Dataset length before anomaly check is {length}'.format(length=data.shape[0]))
data_catched = catch_overflow(data, param_limits)
print('Dataset length after anomaly check is {length}'.format(length=data_catched.shape[0]))
data_catched.describe()

Dataset length before anomaly check is 14
Dataset length after anomaly check is 14


,Beam length (um),Beam width (nm),Thickness_1 (nm),Thickness_2 (nm),Temperature (K),Distance (nm),Gate voltage (V),M1 Eigenfrequency (Hz),M1 Quality factor,M1 Effective mass (kg),...,M3 Eigenfrequency (Hz),M3 Quality factor,M3 Effective mass (kg),M3 Noise (kg^2/s^3),M3 TED (W),M4 Eigenfrequency (Hz),M4 Quality factor,M4 Effective mass (kg),M4 Noise (kg^2/s^3),M4 TED (W)
count,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,1.400000e+01,14.000000,1.400000e+01,...,1.400000e+01,14.000000,1.400000e+01,1.400000e+01,1.400000e+01,1.400000e+01,14.000000,1.400000e+01,1.400000e+01,1.400000e+01
mean,1034.714286,315.601504,256.578947,37.462406,177.071767,594.323308,35.413575,3.227279e+08,18923.601394,2.413237e-13,...,3.405619e+09,18221.225069,2.502773e-13,2.688209e-18,6.150541e-11,4.929132e+09,18099.017534,2.531605e-13,1.620919e-19,6.561146e-11
std,1088.616160,143.067161,111.274458,17.485986,95.374928,314.747754,19.075590,7.546824e+08,8567.713779,2.811515e-13,...,6.766423e+09,9698.334858,2.919512e-13,1.005259e-17,1.981199e-10,1.016192e+10,9855.537921,2.959491e-13,6.045899e-19,1.953438e-10
min,1.000000,50.000000,50.000000,5.000000,0.010000,10.000000,0.000100,1.699425e+04,3000.000083,4.463888e-18,...,3.596841e+04,1.016647,1.484737e-18,2.328940e-27,9.672427e-13,4.514468e+04,16.412072,3.025101e-19,2.924100e-27,4.907778e-13
25%,22.000000,198.026316,165.131579,23.092105,98.690921,335.657895,19.736909,4.038836e+04,11881.005880,1.954371e-15,...,1.019200e+05,10391.873758,1.744256e-15,6.227859e-27,3.937172e-12,1.264411e+05,10467.563120,1.972066e-15,8.313482e-27,6.454650e-12
50%,725.000000,346.052632,280.263158,41.184211,197.371842,661.315789,39.473718,1.145266e+05,20762.851195,1.226937e-13,...,3.446291e+05,20761.811166,1.227088e-13,4.535756e-26,6.811453e-12,4.607318e+05,20763.196185,1.227512e-13,6.067718e-26,1.121587e-11
75%,1862.500000,423.026316,340.131579,50.592105,248.685921,830.657895,49.736859,3.471756e+08,25384.544122,4.655202e-13,...,4.768071e+09,25380.973552,4.655528e-13,5.820432e-24,1.081437e-11,5.742563e+09,25380.297281,4.655935e-13,5.794502e-24,1.751384e-11
max,3000.000000,500.000000,400.000000,60.000000,300.000000,1000.000000,60.000000,2.803040e+09,29834.677320,8.274242e-13,...,2.368622e+10,29887.471445,8.278642e-13,3.761489e-17,7.492188e-10,3.611185e+10,29895.736550,8.281712e-13,2.262673e-18,7.428655e-10


# Добавление вклада термоупругих потерь

Подмешаем к добротности в датасете добротность согласно приближению Roukes:

In [ ]:
# Fundamental properties:
rho = [3100, 2700] # density of matter
E = [250e9, 70e9] # Young's modulus
k = [20, 237] # Thermal conductivity constant
alpha = [2.3e-6, 23.1e-6] # Thermal expansion coefficient

# Calculated properties:
Cp = [700, 904] # heat capacity at constant pressure

In [ ]:
def calculate_Q_TED(df):
    param_history = {'Cp_av': [], 'E_av': [], 'rho_av': [], 'k_av': [], 'alpha_av': [], 'xi_av': [], 'chi': [], 'Q_ted': []}
    for row_index in range(0, df.shape[0]):
        # Calculating average properties of beam:
        Cp_av = (Cp[0] * df.iloc[row_index, 2] + Cp[1] * df.iloc[row_index, 3]) / (df.iloc[row_index, 2] + df.iloc[row_index, 3])
        E_av = (E[0] * df.iloc[row_index, 2] + E[1] * df.iloc[row_index, 3]) / (df.iloc[row_index, 2] + df.iloc[row_index, 3])
        rho_av = (rho[0] * df.iloc[row_index, 2] + rho[1] * df.iloc[row_index, 3]) / (df.iloc[row_index, 2] + df.iloc[row_index, 3])
        k_av = (k[0] * df.iloc[row_index, 2] + k[1] * df.iloc[row_index, 3]) / (df.iloc[row_index, 2] + df.iloc[row_index, 3])
        alpha_av = (alpha[0] * df.iloc[row_index, 2] + alpha[1] * df.iloc[row_index, 3]) / (df.iloc[row_index, 2] + df.iloc[row_index, 3])
        xi_av = k_av / (rho_av * Cp_av)
        param_history['Cp_av'].append(Cp_av)
        param_history['E_av'].append(E_av)
        param_history['rho_av'].append(rho_av)
        param_history['k_av'].append(k_av)
        param_history['alpha_av'].append(alpha_av)
        param_history['xi_av'].append(xi_av)

        for mode_number in range(1, 5):
            # Calculating TED quality factor:
            chi = df.iloc[row_index, 1] * 1e-9 * np.sqrt(2 * 3.1415 * df.loc[df.index[row_index], f'M{mode_number} Eigenfrequency (Hz)'] / (2 * xi_av))
            delta_ted = (E_av * alpha_av **2 * df.iloc[row_index, 4]) / (Cp_av * rho_av) * (6/(chi**2) - 6 * (np.sinh(chi) + np.sin(chi))/(chi**3 * (np.cosh(chi) + np.cos(chi))))
            df.loc[df.index[row_index], f'M{mode_number} Quality factor'] = 1/(1/df.loc[df.index[row_index], f'M{mode_number} Quality factor'] + delta_ted)

            param_history['chi'].append(chi)
            param_history['Q_ted'].append(1/delta_ted)
        
    return df, param_history

In [ ]:
print(data_catched.shape[0])
data_ted, history = calculate_Q_TED(data_catched)

1929


In [ ]:
sorted_history = sorted(history['Q_ted'])
print(sorted_history)

[3704.473112453825, 4064.067581841001, 4655.2362501210455, 5491.329506911857, 6165.336683564043, 6670.803259367736, 7010.671178321441, 7146.545044563837, 7832.391596822633, 8446.139052810784, 9285.450436348534, 9512.741526918864, 9681.64549363439, 10463.03360806882, 10547.584537268762, 11603.575132551907, 11845.058113424247, 11872.331098598659, 13769.263992657461, 14484.717325121825, 14554.509631474637, 14579.379064604884, 14786.283443924349, 14794.266914350384, 14890.51126025794, 15021.574240937796, 15718.510441856246, 16268.407005994397, 16531.50044971198, 17470.69648065002, 17836.005584126568, 18759.27713162145, 19179.084722383483, 19783.683575047806, 19931.43770244576, 21073.027587940185, 22386.29227396868, 22862.094169825617, 23425.832924728966, 24456.365685594814, 24927.31865054894, 25638.77460020904, 25932.742829149905, 26089.212999726715, 26089.339390798104, 26125.853357196327, 26897.69169676202, 26927.720123701263, 27380.54215722304, 27512.333900900485, 27588.51955985044, 2779

In [ ]:
data_ted.head()

,Beam length (um),Beam width (nm),Thickness_1 (nm),Thickness_2 (nm),Temperature (K),Distance (nm),Gate voltage (V),M1 Eigenfrequency (Hz),M1 Quality factor,M1 Effective mass (kg),...,M3 Eigenfrequency (Hz),M3 Quality factor,M3 Effective mass (kg),M3 Noise (kg^2/s^3),M3 TED (W),M4 Eigenfrequency (Hz),M4 Quality factor,M4 Effective mass (kg),M4 Noise (kg^2/s^3),M4 TED (W)
0,821.402113,897.519396,200.0,36.102266,2.248089,924.978487,39.775523,169003.789805,53781.219971,2.581146e-13,...,5.089901e+05,53640.722509,2.582148e-13,4.490996e-28,1.441984e-11,6.809541e+05,53577.353352,2.583005e-13,6.009473e-28,2.286573e-11
1,187.634453,482.065578,150.0,19.450374,161.773060,911.580688,10.599196,512665.917301,27829.840781,2.167964e-14,...,1.616819e+06,25729.966624,2.199621e-14,3.120787e-25,1.475224e-11,2.243525e+06,24670.596346,2.212375e-14,4.356675e-25,1.758884e-11
2,638.204045,242.947138,100.0,24.144951,3.181935,455.886263,15.074379,221111.869936,14575.916394,2.871363e-14,...,6.636297e+05,14574.507206,2.871555e-14,5.640215e-28,7.054148e-13,8.851790e+05,14573.764249,2.871377e-14,7.522689e-28,1.155275e-12
3,910.105786,675.544397,200.0,43.676386,107.099808,914.705274,8.740595,124047.991251,39709.542285,2.224241e-13,...,3.731338e+05,38154.086830,2.224716e-13,1.462375e-26,7.550186e-12,4.986630e+05,37426.945624,2.225221e-13,1.954104e-26,1.220765e-11
4,665.998773,627.258997,200.0,26.364768,0.970622,659.057305,0.787551,201641.652944,37627.851275,1.412254e-13,...,6.068425e+05,37612.298667,1.412640e-13,2.752680e-28,4.751424e-12,8.113547e+05,37608.207889,1.412993e-13,3.680917e-28,1.052563e-11


# Сохраним датасет на гугл-диск:

In [ ]:
%cd /content/drive/My Drive/ML_Data/Data_Processed/
data_ted.to_csv('Data_Processed_5000nm.csv')

/content/drive/My Drive/ML_Data/Data_Processed
